In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from dateutil.relativedelta import *

import os
import re

import matplotlib.pyplot as plt
import seaborn as sns
import graphviz as gr

from scipy import stats
from scipy.interpolate import interp1d

from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tsdisagg import disaggregate_series
from timedisagg.td import TempDisagg

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

import dython
from dython.nominal import associations

In [ ]:
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 3000)

In [ ]:
cur_dir = os.path.abspath(os.path.dirname("__fil__"))

# Data Aggregation and Cleaning

## OECD Files

In [ ]:
def OECD_clean_input(dataframe):
    dataframe = dataframe.astype(str).apply(lambda x: x.str.replace('"', ""))
    dataframe.rename(columns = lambda x: x.replace('"', ""), inplace=True)
    return dataframe

OECDCountryCodeSub = {"DEU":"DE", "FRA":"FR", "SWE":"SE", "ITA":"IT", "GBR":"UK", "DNK":"DK", 
                      "AUT":"AT", "HUN":"HU", "BEL":"BE", "FIN":"FI", "IRL":"IE", "NLD":"NL",
                      "NOR":"NO", "POL":"PL", "PRT":"PT", "CHE":"CH"}

### Projection Integration

#### Absolute Level of Projection is given

##### Total Population

In [ ]:
proj_pop_df = pd.read_csv(os.path.join(cur_dir,"ProjectedPopulation.csv"), quoting=3)
proj_pop_df = OECD_clean_input(proj_pop_df)
proj_pop_df["FREQUENCY"] = "A"

proj_pop_split_df = proj_pop_df[(proj_pop_df["Age"]=="Total") & (proj_pop_df["SEX"]=="T") & (proj_pop_df["TIME"]!="2022")]
proj_pop_split_df["Value"] = proj_pop_split_df["Value"].astype(float)/1000000

pop_df = pd.read_csv(os.path.join(cur_dir,"PopulationLevel.csv"), quoting=3)
pop_df = OECD_clean_input(pop_df)

proj_pop_combine_df = pd.concat([pop_df,proj_pop_split_df]).reset_index(drop=True)
proj_pop_combine_df["Country Code"] = proj_pop_combine_df["LOCATION"].map(OECDCountryCodeSub)

proj_pop_combine_df

##### Elderly Population

In [ ]:
proj_eldpop_split_df = proj_pop_df[(proj_pop_df["Age"]=="Share of 65 and over - elderly") & (proj_pop_df["SEX"]=="T") & (proj_pop_df["TIME"]!="2022")]
   
edlpop_df = pd.read_csv(os.path.join(cur_dir,"ElderlyPopulation.csv"), quoting=3)
edlpop_df = OECD_clean_input(edlpop_df)

proj_eldpop_combine_df = pd.concat([edlpop_df,proj_eldpop_split_df]).reset_index(drop=True)
proj_eldpop_combine_df["Country Code"] = proj_eldpop_combine_df["LOCATION"].map(OECDCountryCodeSub)

proj_eldpop_combine_df

#### Relative Level (Growth Rate) of Projection is given

##### GDP

In [ ]:
#Projection file to get years from 2023 and onwards

proj_gdp_df = pd.read_csv(os.path.join(cur_dir,"ProjectedGDPGrowthRate.csv"), quoting=3)
proj_gdp_df = OECD_clean_input(proj_gdp_df)
proj_gdp_df["Country Code"] = proj_gdp_df["LOCATION"].map(OECDCountryCodeSub)

proj_gdp_re_df = proj_gdp_df[proj_gdp_df["TIME"]!="2022"]

proj_gdp_re_df

In [ ]:
#Historical file to get only 2022 for to be used as base for growth projection

gdp_df = pd.read_csv(os.path.join(cur_dir,"GDP.csv"), quoting=3)
gdp_df = OECD_clean_input(gdp_df)
gdp_df["Country Code"] = gdp_df["LOCATION"].map(OECDCountryCodeSub)

gdp_re_df = gdp_df[gdp_df["TIME"]=='2022']

gdp_re_df

In [ ]:
#Combining historical and projection file with the first value (2022) being actual figure and the remaining figures as projection

proj_gdp_combine_df = pd.concat([gdp_re_df,proj_gdp_re_df]).sort_values(by=["Country Code","TIME"]).reset_index(drop=True)
proj_gdp_combine_df["Value"] = proj_gdp_combine_df["Value"].astype(float)

proj_gdp_combine_country_df =pd.DataFrame()

for country in OECDCountryCodeSub.values():
    country_single = proj_gdp_combine_df[proj_gdp_combine_df["Country Code"]==country].reset_index(drop=True)
    country_single.loc[0,"ProjectedGDPGrowth"] = country_single.loc[0,"Value"]
    for n in range(1, len(country_single)):
        previous = n-1
        country_single.loc[n,"ProjectedGDPGrowth"] = (1+(country_single.loc[n,"Value"]/100))*country_single.loc[previous,"ProjectedGDPGrowth"]
    proj_gdp_combine_country_df = pd.concat([proj_gdp_combine_country_df,country_single])
        
proj_gdp_combine_country_df = proj_gdp_combine_country_df.drop("Value",axis=1).rename(columns={"ProjectedGDPGrowth":"Value"})


#Add back the historical data before year 2022 for interpolation

gdp_re_hist_df = gdp_df[gdp_df["TIME"].astype(int)<2022]
proj_gdp_combine_country_df = pd.concat([gdp_re_hist_df,proj_gdp_combine_country_df]).reset_index(drop=True)
proj_gdp_combine_country_df

#### Time Series Projection

In [ ]:
#Define function for checking stationarity

def stationarity_check(country: [], file, x_axis_name: str):
    palette = sns.color_palette('Set2',len(country))
    country_color_dict = dict(zip(country, palette))
    
    if 'Market EUR' in file.columns:
        file["Country Code"] = file["Country"].map(country_zip)
    else:
        file = OECD_clean_input(file)
        file["Country Code"] = file["LOCATION"].map(OECDCountryCodeSub)

    file_time = file.copy()
    file_time['Value'] = file_time['Value'].astype(float)
    file_time['TIME'] = pd.to_datetime(file_time['TIME'], exact=False)
    file_time.set_index("TIME", inplace=True)
    file_time = file_time[["Country Code","Value"]]

    plt.figure(figsize=(10,5))

    for c in country:
        if 'Market EUR' in file.columns:
            file_time_country = file_time[file_time['Country Code']==c].resample('MS').asfreq().dropna()
        else:
            file_time_country = file_time[file_time['Country Code']==c].resample('YS').asfreq().dropna()
        adf, pvalue, usedlag_, nobs_, critical_values_, icbest_ = adfuller(file_time_country['Value'])
        
        if pvalue > 0.05:
            print('{} - Non-Stationary: {:.3f}'.format(c, pvalue))
        else:
            print('{} - Stationary: {:.3f}'.format(c, pvalue))

        sns.lineplot(x=file_time_country.index, y="Value", data=file_time_country, c=country_color_dict[c])

    plt.ylabel(ylabel=x_axis_name)
    plt.xlabel(xlabel='Year')
    legend_handles = [plt.Line2D([0,1],[1,1], label=c, color=country_color_dict[c], lw=2) for i, c in enumerate(country)]
    plt.legend(handles=legend_handles, title='Countries',loc='center left', fontsize=8, bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()
    return file_time


In [ ]:
#Time Series Training and Prediction

def time_series_proj(country:[],proj_file:[[]],title_name:str):

    para_arima = np.arange(0,6,1)
    proj_file_proj = pd.DataFrame()
    fig,ax = plt.subplots(8,2,figsize=(16,30))

    for i, c in enumerate(country):

        #fix to yearly frequency
        proj_file_country = proj_file[proj_file['Country Code']==c].drop('Country Code', axis=1)
        
        freq_check = len(pd.date_range(proj_file_country.index[-2], proj_file_country.index[-1], freq='MS'))
        
        if freq_check==2:
            proj_file_country = proj_file_country.resample('MS').asfreq()
        elif freq_check==4:
            oidx = proj_file_country.index
            nidx = pd.date_range(oidx.min(), oidx.max(), freq='Q')
            proj_file_country = proj_file_country.reindex(oidx.union(nidx)).rename(columns={'Value':'Old Value'})
            proj_file_country['Value'] = proj_file_country['Old Value'].shift(-1)
            proj_file_country= proj_file_country.drop('Old Value',axis=1).dropna()
        else:
            proj_file_country = proj_file_country.resample('YS').asfreq()
            
        #check for optimal parameters
        arima_model = auto_arima(proj_file_country, start_p=para_arima[0], d=para_arima[0], start_q=para_arima[0],
                            max_p=para_arima[len(para_arima)-1], max_d=para_arima[len(para_arima)-1], max_q=para_arima[len(para_arima)-1],
                            start_P=para_arima[0], D=para_arima[0], start_Q=para_arima[0],
                            max_P=para_arima[len(para_arima)-1], max_D=para_arima[len(para_arima)-1], max_Q=para_arima[len(para_arima)-1],
                            stepwise=False)

        #split the data to train and test set
        size = int(len(proj_file_country)*0.8)
        X_train, X_test = proj_file_country[0:size], proj_file_country[size:len(proj_file_country)]

        #train the model with data training set
        if arima_model.get_params()['seasonal_order'][-1] > 1:
            model = SARIMAX(X_train, 
                            order=arima_model.get_params()['order'], 
                            seasonal_order=arima_model.get_params()['seasonal_order'],
                            initialization='approximate_diffuse')
        else:
            model = SARIMAX(X_train, 
                            order=arima_model.get_params()['order'],
                            initialization='approximate_diffuse')

        result = model.fit()
        
        
        #number of years to predict
        if freq_check==2:
            year_pred = len(pd.date_range((proj_file_country.index[-1]+relativedelta(months=1)),
                            pd.to_datetime('2024',format='%Y',exact=False),
                            freq='MS'))-1
                
        elif freq_check==4:
            year_pred = len(pd.date_range((proj_file_country.index[-1]+relativedelta(months=3)),
                            pd.to_datetime('2024',format='%Y',exact=False),
                            freq='QS'))-1

        else:
            year_pred = 2024-(proj_file_country.index[-1].year+1)
        

        #check the prediction of both training and testing set
        train_pred = result.predict(0,size)
        test_pred = result.predict(size, len(proj_file_country)+year_pred)
        

        #plot the prediction

        ax[i][0].plot(train_pred, label= 'Training Set Prediction')
        ax[i][0].plot(proj_file_country['Value'], label= 'Actual Figures')
        ax[i][0].plot(test_pred, label= 'Testing Set Prediction')
        ax[i][0].set_title('{} - Prediction with Splitting Dataset'.format(c))
        ax[i][0].legend()


        #train the model without splitting the data as seems like to more accurate for only predicting one year after
        #approximate_diffuse to avoid a possible error called LU Decomposition, not sure if there is any side-effect
        if arima_model.get_params()['seasonal_order'][-1] > 1:
            model = SARIMAX(proj_file_country, 
                            order=arima_model.get_params()['order'], 
                            seasonal_order=arima_model.get_params()['seasonal_order'],
                            initialization='approximate_diffuse')
        else:
            model = SARIMAX(proj_file_country, 
                            order=arima_model.get_params()['order'],
                            initialization='approximate_diffuse')

        result = model.fit()

        #check the prediction of both training and testing set
        train_pred = result.predict(0,size)
        test_pred = result.predict(size, len(proj_file_country)+year_pred)

        #plot the prediction

        ax[i][1].plot(train_pred, label= 'Training Set Prediction')
        ax[i][1].plot(proj_file_country['Value'], label= 'Actual Figures')
        ax[i][1].plot(test_pred, label= 'Testing Set Prediction')
        ax[i][1].set_title('{} - Prediction without Splitting Dataset'.format(c))
        ax[i][1].legend()

        #include the prediction value in the dataset
        pred_df = pd.DataFrame(test_pred[-(year_pred+1):]).rename(columns={'predicted_mean':'Value'})
        proj_file_country = pd.concat([proj_file_country,pred_df])
        proj_file_country['Country Code'] = c

        #concat all the individual back to one big table
        proj_file_proj = pd.concat([proj_file_proj,proj_file_country])

    plt.title(title_name)
    plt.tight_layout()
    plt.show()
    return proj_file_proj

## Disaggregation

In [ ]:
def proj_disagg(country:[], agg_fil:[[]], col_name:str):
    
    final_proj_disagg = pd.DataFrame()
    fig,ax = plt.subplots(2,4,figsize=(30,16))
    
    for i, c in enumerate(country):
        
        agg_fil_new = agg_fil.copy()
        
        if col_name in ts_proj:
            agg_fil_new = agg_fil_new.reset_index()
            agg_fil_new['TIME'] = pd.to_datetime(agg_fil_new['index'].astype(str)).dt.to_period('Y').dt.to_timestamp(how='end')
            
        else:
            agg_fil_new['TIME'] = pd.to_datetime(agg_fil_new['TIME']+'-12-31',exact=False)
            
        agg_fil_new = agg_fil_new[agg_fil_new['Country Code']==c].set_index('TIME')[['Value']]
        agg_fil_new['Value'] = agg_fil_new['Value'].astype(float)

        oidx = agg_fil_new.index
        nidx = pd.date_range(oidx.min(), oidx.max(), freq='M')
        res = agg_fil_new.reindex(oidx.union(nidx)).interpolate('cubicspline')
        
        res['Country Code'] = c
        final_proj_disagg = pd.concat([final_proj_disagg,res])
        
        ax.flatten()[i].plot(res['Value'], label= 'Disaggregated Level')
        ax.flatten()[i].plot(agg_fil_new['Value'], label= 'Actual Level')
        ax.flatten()[i].set_title('{} - {} Disaggregated Time Series'.format(c, col_name))
        ax.flatten()[i].legend()
        
    plt.show()
    display(final_proj_disagg)
    print('-'*122)
    
    final_proj_disagg = final_proj_disagg.reset_index().rename(columns={'Value':col_name})
    final_proj_disagg['Year'] = final_proj_disagg['index'].dt.year
    final_proj_disagg['Month'] = final_proj_disagg['index'].dt.month
        
    return final_proj_disagg

### Read all and combine all files

In [ ]:
folder_path = os.path.join(cur_dir,"OECD")

files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

PreAvoidDup = "1"
Agg_OECD_df = Agg_df.copy()

for file in files:
    file_path = os.path.join(folder_path, file)
    
    if file_path.lower().endswith("csv"):
        try:
            if 'actual' in file_path.lower():
                content = pd.read_csv(file_path, quoting=3)
                Agg_OECD_df = pd.merge(Agg_OECD_df, content.drop('index',axis=1), 
                                  on=["Country Code", "Year", "Month"], how="left")
                
            else:
                content = pd.read_csv(file_path, quoting=3)
                content = OECD_clean_input(content)
                content["Value"] = content["Value"].astype(float)
                content["Country Code"] = content["LOCATION"].map(OECDCountryCodeSub)
                content["Year_Month"] = content["TIME"]
                Agg_OECD_df = pd.merge(Agg_OECD_df, content[["Country Code", "Year_Month", "Value"]], 
                                  on=["Country Code", "Year_Month"], how="left")

                Agg_OECD_df.rename(columns={"Value":"Value"+PreAvoidDup}, inplace=True)

                PreAvoidDup += "1"            
                        
        except pd.errors.ParserError as e:
            print(f"Error parsing {file}: {e}")
            
    else:
        print(f"Skipping non-csv file: {file}")
        
OldColumnName = Agg_OECD_df.columns.tolist()
NewColumnName = ['Year_Month', 'Year', 'Month', 'Quarter', 'Country', 'Country Code', 'FiscalYear', 'FiscalMonth', 
                 'FiscalYearMonth', 'Average Regulatory Index Score', 'Household Disposable Income per Capita', 
                 'Elderly Population Proportion', 'GDP (in millions)', 'Healthcare Spending Proportion',
                 'Parmaceutical Spending Proportion', 'Population Level', 'Tertiary Education Level Proportion',
                 'Inflation Rate', 'Interest Rate', 'Unemployment Rate']
old_new_col_name_map = dict(zip(OldColumnName, NewColumnName))

Agg_OECD_df.rename(columns=old_new_col_name_map, inplace=True)

Agg_OECD_df["Elderly Population Proportion"] = Agg_OECD_df.apply(lambda df: df["Elderly Population Proportion"]*1000000 if df["Elderly Population Proportion"]<0.001
                                                      else df["Elderly Population Proportion"], axis=1)

Agg_OECD_df

## Financial Statement Data

In [ ]:
#Financial data function for reading IS and BS

def fin_data_read(folder_path:str, data_needed:str, first_file_toread:str, skiprows:int, usecols:[], filename:str,) -> pd.DataFrame:

    folder_path = folder_path

    files = [f for f in os.listdir(folder_path)]

    sheet_pattern = re.compile(r"^(?:Company_)?[A-Za-z]{{2}}{}$".format(data_needed))

    fin_df = pd.DataFrame({"YearMonth_Country_FinItems":["Year_Month","Country"]})
    sup_df = pd.read_excel(first_file_toread, 
                           sheet_name="Company{}".format(data_needed), skiprows=skiprows, usecols=[0])
    sup_df.columns = fin_df.columns
    fin_df = pd.concat([fin_df, sup_df], ignore_index=True)

    Avoid_Dup = "1"

    for file in files:
        file_path = os.path.join(folder_path, file)

        if file.upper().startswith(filename):
            file_ext_name = os.path.basename(file_path)
            file_year_month = file_ext_name[-9:-5]

            file_df = pd.ExcelFile(file_path)
            all_sheet_names = file_df.sheet_names

            for sheet_name in all_sheet_names:
                if sheet_name != "Company_BS":
                    if sheet_pattern.match(sheet_name):
                        file_country = sheet_name
                        sup_data = file_df.parse(sheet_name, skiprows=skiprows, usecols=usecols)
                        sup_data.columns = ["YearMonth_Country_FinItems","YearMonth_Country_FinData"+Avoid_Dup]

                        month_data = pd.DataFrame({"YearMonth_Country_FinItems":["Year_Month","Country"],
                                                  "YearMonth_Country_FinData"+Avoid_Dup:[file_year_month,file_country]})

                        fin1_df = pd.concat([month_data,sup_data], ignore_index=True)
                        try:
                            fin1_df.iloc[9,0] = fin1_df.iloc[9,0].replace("Payments","Payments_sub")

                        except:
                            pass
                        fin_df = pd.merge(fin_df, fin1_df, on="YearMonth_Country_FinItems", how="outer")
                        fin_df = fin_df.dropna(subset=["YearMonth_Country_FinItems"], how="all")

                        Avoid_Dup += "1"

    return fin_df


## Datahub

#### Price Range Categorization

In [ ]:
price_quantiles = pd.qcut(outbound_excl_cus["OrderPriceSales"], q=[0,0.2,0.4,0.6,0.8,1.0], labels=["Very Low","Low", "Medium", "High", "Very High"])

outbound_price_range = outbound_excl_cus.loc[:]
outbound_price_range["Price Categorization"] = price_quantiles

outbound_price_range = outbound_price_range.groupby(["Year_Month","Country Code","Price Categorization"])["Deliver Export Turnover"].sum().reset_index()
outbound_tot_price = outbound_price_range.groupby(["Year_Month","Country Code"])["Deliver Export Turnover"].transform("sum")

outbound_price_range["Price Range Revenue Proportion"] = outbound_price_range["Deliver Export Turnover"]/outbound_tot_price

outbound_price_range_pivot = outbound_price_range.drop("Deliver Export Turnover", axis=1).pivot_table(index=["Year_Month","Country Code"],
                                                                                                     columns =["Price Categorization"],
                                                                                                     values=["Price Range Revenue Proportion"],
                                                                                                     aggfunc="sum",
                                                                                                     fill_value=0)

outbound_price_range_pivot.columns = [f"{col[1]}_{col[0]}" for col in outbound_price_range_pivot.columns]
outbound_price_range_pivot.reset_index(inplace=True)

outbound_price_range_pivot

# Exploratory Data Analysis

### All Features

#### PCA Analysis

In [ ]:
num_transformer = Pipeline(steps=[('NaNnum',SimpleImputer(strategy='mean')),
                                  ('scale',StandardScaler())])

bi_cat_transformer = Pipeline(steps=[('NaNcat',SimpleImputer(strategy='most_frequent')),
                                  ('onehot',OneHotEncoder())])

preprocessor_pca = ColumnTransformer(transformers=[('num_transformer',num_transformer,target_features+numeric_features),
                                               ('bi_cat_transformer',bi_cat_transformer,binary_categorical_features)])

pipeline_pca = Pipeline(steps=[('preprocessor',preprocessor_pca),
                           ('pca',PCA(n_components=2))])

pca = pipeline_pca.fit_transform(Agg_df_infadj[target_features+numeric_features+binary_categorical_features])

In [ ]:
palette = sns.color_palette('Set2',len(country))
country_color_dict = dict(zip(country,palette))
country_color = [country_color_dict[x] for x in Agg_df_infadj["Country"]]

for sample in range(len(pca)):
    plt.scatter(pca[sample][0],pca[sample][1],color=country_color[sample],s=8)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('PCA Analysis')
legend_handles = [plt.Line2D([0],[0], marker='o', color='w', label=label,
                             markerfacecolor=palette[i], markersize=8) for i, label in enumerate(country)]
plt.legend(handles=legend_handles, title='Countries',loc='center left', fontsize=8, bbox_to_anchor=(1, 0.5))
plt.show()

#### Numeric Features

In [ ]:
#Summary statistics for numeric features

summary_stat_num = Agg_df_infadj[target_features+numeric_features].describe()
summary_stat_num.loc["+3_std"] = summary_stat_num.loc["mean"]+(summary_stat_num.loc["std"]*3)
summary_stat_num.loc["-3_std"] = summary_stat_num.loc["mean"]-(summary_stat_num.loc["std"]*3)

summary_stat_num

In [ ]:
#Possible outliers outside of +/-3 std
pos_out_num = Agg_df_infadj[(np.abs(stats.zscore(Agg_df_infadj[target_features+numeric_features]))>3).any(axis=1)]
pos_out_num

In [ ]:
#Correlation Matrix
corr_num = Agg_df_infadj[target_features+numeric_features].corr()

plt.figure(figsize=(15,15))
sns.heatmap(corr_num, xticklabels=corr_num.columns, yticklabels=corr_num.columns, cmap="RdBu")
plt.show()

display(corr_num)

In [ ]:
pd.plotting.scatter_matrix(corr_num, alpha=1, figsize=(100,100))

#### Categorical Features

In [ ]:
summary_stat_bi_cat = Agg_df_infadj[binary_categorical_features].describe(include=["category","object"])

summary_stat_bi_cat

In [ ]:
#from github (dython)
associations(Agg_df_infadj[binary_categorical_features], nom_nom_assoc="theil", figsize=(15, 15), cmap='RdBu')["corr"]


## Main Features

In [ ]:
#from github (dython)
associations(Agg_df_infadj[target_features+main_numeric_features+main_binary_categorical_features], nom_num_assoc="correlation_ratio", figsize=(15, 15), cmap='RdBu')['corr']


### Univariate Analysis

#### Numeric main Features

In [ ]:
#Not transformed with log

fig, axes = plt.subplots(6, 4, figsize=(25,20))

for i, ax in enumerate(axes.flatten()):
    if i < len(target_features+main_numeric_features):
        sns.histplot(x=Agg_df_infadj[target_features+main_numeric_features].iloc[:,i], ax=ax)
        ax.axvline(Agg_df_infadj[target_features+main_numeric_features].iloc[:,i].mean(), color='magenta', linestyle='dashed', linewidth=2)
        ax.axvline(Agg_df_infadj[target_features+main_numeric_features].iloc[:,i].median(), color='cyan', linestyle='dashed', linewidth=2)
        ax.set_title(Agg_df_infadj[target_features+main_numeric_features].columns[i])
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    else:
        ax.axis('off')

fig.suptitle("Univariate Analysis of Key Numeric Features", fontsize='x-large')       
        
plt.show()

In [ ]:
#Transformed with log

fig, axes = plt.subplots(6, 4, figsize=(25,20))

for i, ax in enumerate(axes.flatten()):
    if i < len(target_features+main_numeric_features):
        sns.histplot(x=np.log1p(Agg_df_infadj[target_features+main_numeric_features]).iloc[:,i], ax=ax)
        ax.axvline(np.log1p(Agg_df_infadj[target_features+main_numeric_features]).iloc[:,i].mean(), color='magenta', linestyle='dashed', linewidth=2)
        ax.axvline(np.log1p(Agg_df_infadj[target_features+main_numeric_features]).iloc[:,i].median(), color='cyan', linestyle='dashed', linewidth=2)
        ax.set_title(Agg_df_infadj[target_features+main_numeric_features].columns[i])
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    else:
        ax.axis('off')

fig.suptitle("Univariate Analysis of Key Numeric Features (Log Transformed)", fontsize='x-large')       
        
plt.show()

#### Categorical main Features

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10), gridspec_kw={'hspace':0.6})

for i, ax in enumerate(axes.flatten()):
    if i < 2:
        sns.countplot(x=Agg_df_infadj[main_binary_categorical_features].iloc[:,i], ax=ax, palette='Blues')
        ax.set_title(Agg_df_infadj[main_binary_categorical_features].columns[i])
        ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    else:
        sns.countplot(x=Agg_df_infadj[main_binary_categorical_features].iloc[:,i], ax=ax, palette='Blues')
        ax.set_title(Agg_df_infadj[main_binary_categorical_features].columns[i])
        ax.set_xlabel(xlabel='')
        ax.set_ylabel(ylabel='')
    
fig.suptitle("Univariate Analysis of Key Binary and Categorical Features",fontsize='x-large')

plt.tight_layout()
plt.show()

### Bivariate Analysis

#### Numeric main Features

In [ ]:
sns.pairplot(Agg_df_infadj, vars=target_features+main_numeric_features, hue='Country', corner=True, palette='Set2')

#### Categorical main Features

In [ ]:
fig, axes = plt.subplots(2,4,figsize=(35,15),sharey='row', sharex='col')


for t_i, target in enumerate(target_features):
    for p_i, pred in enumerate(main_binary_categorical_features):
        if t_i==0 and p_i==0:
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            ax.set_xlabel(xlabel='')
            handles, labels = ax.get_legend_handles_labels()
            ax.get_legend().remove()
            
        elif t_i==1 and p_i==0:
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
            ax.get_legend().remove()
            
        elif t_i==1 and p_i in range(1,len(main_binary_categorical_features)):
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            if p_i == 1:
                ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
            ax.set_ylabel(ylabel='')
            ax.get_legend().remove()
        
        else:
            ax=axes[t_i][p_i]
            sns.boxplot(Agg_df_infadj,
                x=pred, 
                y=target, 
                hue="Country", 
                palette='Set2',
                ax=ax)
            ax.set_xlabel(xlabel='')
            ax.set_ylabel(ylabel='')
            ax.get_legend().remove()
            
fig.legend(handles,labels,loc='outside right center', title="Country")

plt.tight_layout()
plt.show()

## Feature Importance (Random Forest)

In [ ]:
####Train the model

folds = 5
cv = KFold(n_splits=folds-1)
param_grid = {'rf__max_depth': [10,20,30,40,50],
              'rf__n_estimators': np.arange(50,350,50)}

preprocessor_rf = ColumnTransformer(transformers=[('num_transformer',num_transformer,numeric_features),
                                               ('bi_cat_transformer',bi_cat_transformer,binary_categorical_features)])

pipeline_rf = Pipeline(steps=[('preprocessor',preprocessor_rf),
                           ('rf',RandomForestRegressor(n_estimators=100, random_state=0))])


GSCV_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=cv)

X = Agg_df_infadj[numeric_features+binary_categorical_features]

for i in target_features:
    y = SimpleImputer(strategy='mean').fit_transform(Agg_df_infadj[[i]]).reshape(-1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/folds, random_state=0)

    GSCV_rf.fit(X_train,y_train)

    
    ####Model Evaluation and Prediction

    print('GridSearchCV Random Forest NMAE Best Score for Target Feature - {}: {:.3f}'.format(i,GSCV_rf.best_score_))
    print('GridSearchCV Random Forest NMAE Best Score for Target Feature - {}: {}'.format(i,GSCV_rf.best_params_))
    print('-'*100)

    y_pred = GSCV_rf.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print('MSE - {}: {:.3f}'.format(i,mse))
    print('RMSE - {}: {:.3f}'.format(i,rmse))
    print('R2 - {}: {:.3f}'.format(i,r2))
    print('-'*100)

    plt.scatter(y_test, y_pred)
    plt.xlabel('Actual Value')
    plt.ylabel('Predicted Value')
    plt.title('Random Forest Predictions (All Features) - {}'.format(i))
    coeff = np.polyfit(y_test,y_pred,1)
    p_line = np.poly1d(coeff)
    plt.plot(y_test,p_line(y_test),color='darkorange')

    plt.show()

    
    ####Feature Importance (MDI - Impurity)

    rf_feat_name = [name.split('__')[1] for name in GSCV_rf.best_estimator_[:-1].get_feature_names_out()]
    rf_feat_imp = GSCV_rf.best_estimator_[-1].feature_importances_

    feat_imp_df = pd.Series(rf_feat_imp, index=rf_feat_name).sort_values(ascending=False)

    plt.figure(figsize=(20,20))
    sns.barplot(y=feat_imp_df.index,x=feat_imp_df,palette='Blues_r')
    plt.title('Random Forest Feature Importance (MDI) - {}'.format(i),fontsize='xx-large')
    plt.tight_layout()
    plt.show()
    
    
    ####Feature Importance (MDA - Accurcy - Permutation)
    
    #On training data
    feat_MDA_result = permutation_importance(
        GSCV_rf, X_train, y_train, n_repeats=10, random_state=0, n_jobs=2)

    sorted_importance_idx = feat_MDA_result.importances_mean.argsort()

    feat_imp_pi_df = pd.DataFrame(feat_MDA_result.importances[sorted_importance_idx].T,
                                  columns=X.columns)

    plt.figure(figsize=(20,20))
    sns.boxplot(feat_imp_pi_df,orient='h',palette='Blues')
    plt.axvline(x=0, color="darkorange", linestyle="--", linewidth=2)
    plt.title('Random Forest Feature Importance_Training Set (MDA) - {}'.format(i),fontsize='xx-large')
    plt.tight_layout()
    plt.show()
    
    
    #On testing data
    feat_MDA_result = permutation_importance(
        GSCV_rf, X_test, y_test, n_repeats=10, random_state=0, n_jobs=2)

    sorted_importance_idx = feat_MDA_result.importances_mean.argsort()

    feat_imp_pi_df = pd.DataFrame(feat_MDA_result.importances[sorted_importance_idx].T,
                                  columns=X.columns)

    plt.figure(figsize=(20,20))
    sns.boxplot(feat_imp_pi_df,orient='h',palette='Blues')
    plt.axvline(x=0, color="darkorange", linestyle="--", linewidth=2)
    plt.title('Random Forest Feature Importance_Test Set (MDA) - {}'.format(i),fontsize='xx-large')
    plt.tight_layout()
    plt.show()

# DAG Analysis

In [ ]:
#Show different types of Basic DAGs

DAG_chain = gr.Digraph()
DAG_chain.edge("X", "Y")
DAG_chain.edge("Y", "Z")
DAG_chain

DAG_fork = gr.Digraph()
DAG_fork.edge("X", "Y")
DAG_fork.edge("X", "Z")
DAG_fork

DAG_collider = gr.Digraph()
DAG_collider.edge("X", "Z")
DAG_collider.edge("Y", "Z")
DAG_collider

In [ ]:
#Save the DAGs

DAG_chain.render("Chain_DAG",view=True,format="png")
DAG_fork.render("Fork_DAG",view=True,format="png")
DAG_collider.render("Collider_DAG",view=True,format="png")

In [ ]:
#Backdoor Adjustment

DAG_b4Backdoor = gr.Digraph()
DAG_b4Backdoor.edge("X", "Y")
DAG_b4Backdoor.edge("X", "Z")
DAG_b4Backdoor.edge("Y", "Z")
DAG_b4Backdoor

DAG_afterBackdoor = gr.Digraph()
DAG_afterBackdoor.edge("X", "Z")
DAG_afterBackdoor.edge("Y", "Z")
DAG_afterBackdoor

In [ ]:
#Save the DAGs

DAG_b4Backdoor.render("b4Backdoor_DAG",view=True,format="png")
DAG_afterBackdoor.render("afterBackdoor_DAG",view=True,format="png")